# Hypothesis and Inference

Often, as data scientists, we’ll want to test whether a certain hypothesis is likely to be true.
For our purposes, hypotheses are assertions like “this coin is fair” or “data scientists prefer
Python to R”

### Flipping a Coin

Imagine we have a coin and we want to test whether it's fiar.

In [1]:
import math

In [2]:
def normal_approximation_to_binomial(n, p):
    """finds mu and sigma corresponding to a Binomial(n, p)"""
    mu = n * p
    sigma = math.sqrt(p * (1 - p) * n)
    return mu,sigma